In [27]:
import constants
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [28]:
myCmap=sns.diverging_palette(170, 330, l=65, center="dark", as_cmap=True)
physical = {constants.SHOCK: ['Time', 'Density', 'gasTemp', 'av', 'zeta', 'radfield', constants.SHOCKVEL],
            constants.HOTCORE: ['Time', 'Density', 'gasTemp', 'av', 'zeta', 'radfield']}
species=['#CH3OH', 'CH3OH', '#SIO', 'SIO']

In [48]:
tipo= constants.HOTCORE
df= pd.read_csv(tipo+'.csv', index_col=0)
df= df.loc[df.loc[:,species].ge(1e-14).all(axis=1)]

In [49]:
df

,Time,Density,gasTemp,av,zeta,point,HSIO,H,#H,H+,...,@HS2,H2S2+,H2S2,#H2S2,@H2S2,E-,BULK,SURFACE,radfield,runName
9,10.0,100000000.0,15.05,96440.00,1000.0,1,7.732370e-08,0.000003,2.856890e-18,8.306270e-10,...,7.313340e-13,8.596660e-16,3.769300e-13,2.077730e-11,4.238350e-09,8.281280e-09,0.000380,0.000005,1000.0,1000.0_1000.0_100000000.0_300.0.dat
10,100.0,100000000.0,15.37,96440.00,1000.0,1,7.732370e-08,0.000003,3.690720e-19,2.630980e-10,...,7.282710e-13,1.450750e-15,1.134820e-12,3.236040e-11,4.220600e-09,6.047940e-09,0.000379,0.000005,1000.0,1000.0_1000.0_100000000.0_300.0.dat
11,1000.0,100000000.0,17.59,96440.00,1000.0,1,7.732380e-08,0.000003,9.408580e-20,3.147690e-10,...,7.052800e-13,7.743470e-16,2.950460e-12,5.459740e-11,4.087360e-09,4.531750e-08,0.000367,0.000005,1000.0,1000.0_1000.0_100000000.0_300.0.dat
12,10000.0,100000000.0,32.89,96440.00,1000.0,1,7.732450e-08,0.000003,6.759900e-21,3.707380e-11,...,4.938130e-13,2.845820e-16,1.808040e-11,5.156940e-11,2.861830e-09,1.776570e-07,0.000257,0.000005,1000.0,1000.0_1000.0_100000000.0_300.0.dat
13,10100.0,100000000.0,33.04,96440.00,1000.0,1,7.732450e-08,0.000003,1.077400e-23,4.175740e-11,...,4.859680e-13,4.782920e-16,2.567690e-11,7.758770e-11,2.816360e-09,2.233230e-07,0.000253,0.000005,1000.0,1000.0_1000.0_100000000.0_300.0.dat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,960100.0,1000000.0,50.00,966.37,10.0,1,7.304470e-09,0.000003,4.047940e-26,1.570710e-11,...,3.571360e-11,2.610830e-15,2.652790e-09,2.429270e-09,2.093800e-08,3.563330e-07,0.000092,0.000005,1000.0,10.0_1000.0_1000000.0_50.0.dat
6991,970100.0,1000000.0,50.00,966.37,10.0,1,7.304540e-09,0.000003,4.043900e-26,1.557000e-11,...,3.467070e-11,2.598870e-15,2.667520e-09,2.426760e-09,2.032650e-08,3.575800e-07,0.000089,0.000005,1000.0,10.0_1000.0_1000000.0_50.0.dat
6992,980100.0,1000000.0,50.00,966.37,10.0,1,7.304620e-09,0.000003,4.039860e-26,1.543610e-11,...,3.362890e-11,2.587120e-15,2.682090e-09,2.424260e-09,1.971580e-08,3.588170e-07,0.000086,0.000005,1000.0,10.0_1000.0_1000000.0_50.0.dat
6993,990100.0,1000000.0,50.00,966.37,10.0,1,7.304690e-09,0.000003,4.035830e-26,1.530520e-11,...,3.258810e-11,2.575550e-15,2.696500e-09,2.421750e-09,1.910560e-08,3.600440e-07,0.000084,0.000005,1000.0,10.0_1000.0_1000000.0_50.0.dat
